In [1]:
import os
import pandas as pd
from selenium import webdriver
import urllib
from urllib.parse import urlparse
from config import *
from word_utils import *
from page_analyser import page_analyser

In [21]:
from selenium.common.exceptions import StaleElementReferenceException

In [2]:
def result_tag2url(rtag):
    return str(rtag.find_element_by_tag_name("a").get_attribute("href"))

In [3]:
wd = webdriver.Chrome(executable_path=os.path.expanduser(webdriver_path))

In [4]:
keywords = ["hello","world"]

In [5]:
table_dir = "data/%s/" % "/".join(keywords)

In [6]:
if not os.path.exists(table_dir):
    os.makedirs(table_dir)

In [7]:
query = urllib.parse.quote(" ".join(keywords))

In [8]:
base_url = "https://www.google.com/search?q="

In [9]:
wd.get(base_url +query)

In [10]:
search_results_table = wd.find_element_by_id("search")

In [11]:
search_results = search_results_table.find_elements_by_class_name("r")

In [12]:
result_urls = list(map(result_tag2url,search_results))

In [34]:
result_url = result_urls[3]

In [35]:
wd.get(result_url)

In [36]:
pa = page_analyser(wd)

In [42]:
try:
    pa.load_sentences(tag2sentences)
    res = pa.get_tables(sentences2table)    
    for k,v in res.items():
        d = urlparse(result_url).netloc
        p = urlparse(result_url).path.replace("/",".")
        file_path = ".".join([k,d+p,"csv"])
        v.to_csv(os.path.join(table_dir, file_path))
except StaleElementReferenceException:
    pass